## Problem Set 2

MACS 30250, Dr. Evans

Ruoyun Tan

In [28]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import time
import random
from sklearn.model_selection import train_test_split
import multiprocessing
from dask import compute, delayed
import dask.multiprocessing
import time

### 1.

In [44]:
df = pd.read_csv('data/Auto.csv', na_values='?')
df.dropna(inplace=True)
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [45]:
condition1 = (df['mpg'] >= df['mpg'].median())
df['mpg_high'] = np.where(condition1, 1, 0)
condition2 = (df['origin'] == 1)
df['orgn1'] = np.where(condition2, 1, 0)
condition3 = (df['origin'] == 2)
df['orgn2'] = np.where(condition3, 1, 0)
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,mpg_high,orgn1,orgn2
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,0,1,0
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,0,1,0
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,0,1,0
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,0,1,0
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,0,1,0


In [46]:
X = df[['cylinders', 'displacement', 'horsepower', 'weight','acceleration', 'year', 'orgn1', 'orgn2']].values
y = df['mpg_high'].values

#### (a)

In [47]:
errs = np.zeros(100)
seeds = np.random.choice(1000, 100, replace=False)
start = time.time()

for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state= seeds[i])
    logit_reg = LogisticRegression(solver='lbfgs', max_iter=10000, n_jobs=1)
    logit_model = logit_reg.fit(X_train, y_train)
    y_pred = logit_model.predict(X_test)
    errs[i] = ((y_test - y_pred) ** 2).mean()
    
time_used = time.time() - start

print('The computation takes', time_used, 'seconds')
print('The error rate is', errs.mean())

The computation takes 2.943120002746582 seconds
The error rate is 0.10086956521739131


#### (b)

In [48]:
import multiprocessing

num_cores = multiprocessing.cpu_count()
print('Number of available cores is', num_cores)

Number of available cores is 8


In [49]:
def cal_err(i, seeds, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state= seeds[i])
    logit_reg = LogisticRegression(solver='lbfgs', max_iter=10000, n_jobs=1)
    logit_model = logit_reg.fit(X_train, y_train)
    y_pred = logit_model.predict(X_test)
    
    return ((y_test - y_pred) ** 2).mean()

lazy_values = []
start = time.time()

for i in range(100):
    lazy_values.append(delayed(cal_err)(i, seeds, X, y))

results = compute(*lazy_values, scheduler=dask.multiprocessing.get, num_workers=num_cores)

time_used = time.time() - start

print('The computation takes', time_used, 'seconds')
print('The error rate is', np.mean(results))

The computation takes 0.7833101749420166 seconds
The error rate is 0.10086956521739131


The error rate is the same as part (a).